In [1]:
import os

import itk
from itk import TubeTK as tube

from itkwidgets import view

import numpy as np

In [2]:
InputBaseName = "../Data/SRdown4"

if not os.path.exists('../Data/Results'):
    os.makedirs('../Data/Results')
OutputBaseName = "../Data/Results/SRdown4-out"

InputFilename = InputBaseName+".nrrd"

im = itk.imread(InputFilename, itk.F)

In [3]:
ImageType = itk.Image[itk.F, 3]

spacing = im.GetSpacing()[0]

imArray = itk.GetArrayFromImage(im)
im_min = float(imArray.min())
im_max = float(imArray.max())

print("Spacing:", spacing)
print("Intensity range:", im_min, "-", im_max)

Spacing: 1.0
Intensity range: 0.0 - 45.11833572387695


In [4]:
view(im)

<IPython.core.display.Javascript object>

In [5]:
imMath = tube.ImageMath.New(im)
imMath.IntensityWindow(im_min, im_max, 0.0, 1.0)
imMath.Blur(0.34*spacing)
imBlur = imMath.GetOutput()
imMath.Blur(1.35*spacing)
imMath.AddImages(imBlur, -1, 1)
imDoG = imMath.GetOutput()

imDoGArray = itk.GetArrayFromImage(imDoG)

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


In [6]:
view(imDoGArray)

<IPython.core.display.Javascript object>

In [7]:
numSeeds = 25
seedCoverage = 20
seedCoord = np.zeros([numSeeds,3])
for i in range(numSeeds):
    seedCoord[i] = np.unravel_index(np.argmax(imDoGArray, axis=None), imDoGArray.shape)
    indx = [int(seedCoord[i][0]),int(seedCoord[i][1]),int(seedCoord[i][2])]
    minX = max(indx[0]-seedCoverage,0)
    maxX = min(indx[0]+seedCoverage,imDoGArray.shape[0])
    minY = max(indx[1]-seedCoverage,0)
    maxY = min(indx[1]+seedCoverage,imDoGArray.shape[1])
    minZ = max(indx[2]-seedCoverage,0)
    maxZ = min(indx[2]+seedCoverage,imDoGArray.shape[2])
    imDoGArray[minX:maxX,minY:maxY,minZ:maxZ]=im_min
    indx.reverse()
    seedCoord[:][i] = im.TransformIndexToPhysicalPoint(indx)
print(seedCoord)

[[115.  99.  53.]
 [114.  71. 116.]
 [ 91.  31.  39.]
 [ 10. 150. 247.]
 [132.  20. 154.]
 [138.  29.  41.]
 [ 10.  48. 229.]
 [119. 112. 117.]
 [149. 126. 110.]
 [ 12.  57.  38.]
 [ 69.  38.  46.]
 [172. 201.  83.]
 [144. 200.  96.]
 [217.  50.  40.]
 [178.  36.  54.]
 [ 75. 175.  40.]
 [186. 121. 100.]
 [191. 177.  66.]
 [159. 189.  42.]
 [153. 164. 103.]
 [ 89. 130. 109.]
 [ 77. 180.  16.]
 [215.  40.  97.]
 [ 10. 182. 245.]
 [108. 189. 116.]]


In [8]:
# Manually extract a few vessels to form an image-specific training set
vSeg = tube.SegmentTubes.New(Input=im)
vSeg.SetVerbose(True)
vSeg.SetMinRoundness(0.1)
vSeg.SetMinRidgeness(0.8)
vSeg.SetMinCurvature(0.001)  # This is the most influential variable - depends on intensity range of data
vSeg.SetRadiusInObjectSpace( 0.5 )
vSeg.SetMinLength(300)
for i in range(numSeeds):
    print("**** Processing seed " + str(i) + " : " + str(seedCoord[i]))
    vSeg.ExtractTubeInObjectSpace( seedCoord[i], i )
    
tubeMaskImage = vSeg.GetTubeMaskImage()

**** Processing seed 0 : [115.  99.  53.]
**** Processing seed 1 : [114.  71. 116.]
**** Processing seed 2 : [91. 31. 39.]
**** Processing seed 3 : [ 10. 150. 247.]
**** Processing seed 4 : [132.  20. 154.]
**** Processing seed 5 : [138.  29.  41.]
**** Processing seed 6 : [ 10.  48. 229.]
**** Processing seed 7 : [119. 112. 117.]
**** Processing seed 8 : [149. 126. 110.]
**** Processing seed 9 : [12. 57. 38.]
**** Processing seed 10 : [69. 38. 46.]
**** Processing seed 11 : [172. 201.  83.]
**** Processing seed 12 : [144. 200.  96.]
**** Processing seed 13 : [217.  50.  40.]
**** Processing seed 14 : [178.  36.  54.]
**** Processing seed 15 : [ 75. 175.  40.]
**** Processing seed 16 : [186. 121. 100.]
**** Processing seed 17 : [191. 177.  66.]
**** Processing seed 18 : [159. 189.  42.]
**** Processing seed 19 : [153. 164. 103.]
**** Processing seed 20 : [ 89. 130. 109.]
**** Processing seed 21 : [ 77. 180.  16.]
**** Processing seed 22 : [215.  40.  97.]
**** Processing seed 23 : [ 10

In [9]:
itk.imwrite(tubeMaskImage, OutputBaseName+"-VesselsInitial.mha")

In [10]:
LabelMapType = itk.Image[itk.UC,3]

trMask = tube.ComputeTrainingMask[ImageType,LabelMapType].New()
trMask.SetInput( tubeMaskImage )
trMask.SetGap( 3 )
trMask.SetObjectWidth( 1 )
trMask.SetNotObjectWidth( 1 )
trMask.Update()
fgMask = trMask.GetOutput()

In [11]:
itk.imwrite(fgMask, OutputBaseName+"-VesselsInitialMask.mha")

In [12]:
enhancer = tube.EnhanceTubesUsingDiscriminantAnalysis[ImageType,LabelMapType].New()
enhancer.AddInput(im)
enhancer.SetLabelMap(fgMask)
enhancer.SetRidgeId(255)
enhancer.SetBackgroundId(128)
enhancer.SetUnknownId(0)
enhancer.SetTrainClassifier(True)
enhancer.SetUseIntensityOnly(True)
enhancer.SetUseFeatureMath(True)
enhancer.SetScales([0.75*spacing, 1.5*spacing, 3.0*spacing])
enhancer.Update()
enhancer.ClassifyImages()

In [13]:
imMath.SetInput(enhancer.GetClassProbabilityImage(0))
imMath.Blur(0.5*spacing)
prob0 = imMath.GetOutput()
imMath.SetInput(enhancer.GetClassProbabilityImage(1))
imMath.Blur(0.5*spacing)
prob1 = imMath.GetOutput()

In [14]:
itk.imwrite(prob0, OutputBaseName+"-Prob0.mha")
itk.imwrite(prob1, OutputBaseName+"-Prob1.mha")

In [15]:
imDiff = itk.SubtractImageFilter(Input1=prob0, Input2=prob1)
imDiffArr = itk.GetArrayFromImage(imDiff)
dMax = imDiffArr.max()
imProbArr = imDiffArr / dMax
imVess = itk.GetImageFromArray(imProbArr)
imVess.CopyInformation(im)

In [16]:
itk.imwrite( imVess, OutputBaseName+"-VesselsEnhanced.mha", compression=True)

In [ ]:
imMath = tube.ImageMath.New(imVess)
imMath.MedianFilter(1)
imMath.Threshold(0.000001, 1, 1, 0)
imVessMask = imMath.GetOutputShort()

ccSeg = tube.SegmentConnectedComponents.New(imVessMask)
ccSeg.SetMinimumVolume(100)
ccSeg.Update()
imVessMask = ccSeg.GetOutput()

itk.imwrite(imVessMask,OutputBaseName+"-VesselSeedsInitialMask.mha")
imVessMask = itk.imread(OutputBaseName+"-VesselSeedsInitialMask.mha", itk.F)

In [ ]:
imMath.SetInput(imVess)
imMath.ReplaceValuesOutsideMaskRange(imVessMask,2,99999,0)
imSeeds = imMath.GetOutput()
itk.imwrite(imSeeds,OutputBaseName+"-VesselSeeds.mha")

In [ ]:
imMath.SetInput(imVessMask)
imMath.Threshold(0,1,1,0)
imVessMaskInv = imMath.GetOutput()

distFilter = itk.DanielssonDistanceMapImageFilter.New(imVessMaskInv)
distFilter.Update()
dist = distFilter.GetOutput()

imMath.SetInput(dist)
imMath.Blur(0.4)
tmp = imMath.GetOutput()
imMath.ReplaceValuesOutsideMaskRange(tmp, 0.1, 10, 0)
imSeedsRadius = imMath.GetOutput()

itk.imwrite(imSeedsRadius, OutputBaseName+"-VesselSeedsRadius.mha")

In [ ]:
imMath.SetInput(im)
imMath.ReplaceValuesOutsideMaskRange(imVessMask, 2, 99999, 0)
imMath.Blur(spacing)
imInput = imMath.GetOutput()

itk.imwrite(imInput,OutputBaseName+"-VesselInput.mha")

In [36]:
numSeeds = 500

vSeg = tube.SegmentTubes.New(Input=imInput)
#vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0.000001)
vSeg.SetMinRoundness(0.1)
vSeg.SetMinRidgeness(0.75)
vSeg.SetMinLevelness(0.001)
vSeg.SetRadiusInObjectSpace( spacing )
vSeg.SetBorderInIndexSpace(3)
vSeg.SetSeedMask( imSeeds )
vSeg.SetSeedRadiusMask( imSeedsRadius )
vSeg.SetOptimizeRadius(True)
vSeg.SetSeedMaskMaximumNumberOfPoints(numSeeds)
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedExtractionMinimumProbability(0.5)
vSeg.ProcessSeeds()

In [37]:
tubeMaskImage = vSeg.GetTubeMaskImage()
itk.imwrite(tubeMaskImage,OutputBaseName+"-Vessels"+str(numSeeds)+".mha")

In [38]:
SOWriter = itk.SpatialObjectWriter[3].New()
SOWriter.SetInput(vSeg.GetTubeGroup())
SOWriter.SetBinaryPoints(True)
SOWriter.SetFileName(OutputBaseName+"-Vessels"+str(numSeeds)+".tre")
SOWriter.Update()

In [39]:
# smooth tubes!
TubeMath = tube.TubeMath[3, itk.F].New()
TubeMath.SetInputTubeGroup(vSeg.GetTubeGroup())
TubeMath.SetUseAllTubes()
TubeMath.SmoothTube(4,"SMOOTH_TUBE_USING_INDEX_GAUSSIAN")
TubeMath.SmoothTubeProperty("Radius",2,"SMOOTH_TUBE_USING_INDEX_GAUSSIAN")
tubes = TubeMath.GetOutputTubeGroup()

In [40]:
ConvSurface = tube.WriteTubesAsPolyData.New()
ConvSurface.SetInput(tubes)
ConvSurface.SetFileName(OutputBaseName+"-Vessels"+str(numSeeds)+".vtp")
ConvSurface.Update()